# Set Up Cloud Notebook Environment

This section shows how to initialize and configure a cloud-based Jupyter notebook session. You can use platforms like Google Colab, Azure Notebooks, or JupyterHub.

In [1]:
# Import Essential Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from datetime import datetime, timedelta

# Upload and Access Data Files

You can upload files to your cloud notebook using the upload button or code. For example, in Google Colab:

```python
from google.colab import files
uploaded = files.upload()
```

To read a CSV file:
```python
import pandas as pd
df = pd.read_csv('your_file.csv')
df.head()
```

In [5]:
# Run Basic Python Code
print("Hello, cloud notebook!")

# Test the Tijuana Flats Taco & Chips deal logic

def get_tijuana_flats_deal_status():
    api_url = "https://site.api.espn.com/apis/site/v2/sports/baseball/mlb/teams/tb/schedule"
    try:
        response = requests.get(api_url)
        data = response.json()
        games = data.get("events", [])
        now = datetime.utcnow()
        # Find the most recent regular season home game within last 7 days
        recent_game = None
        for g in reversed(games):
            competitions = g.get("competitions", [{}])
            competitors = competitions[0].get("competitors", [{}])
            is_home = competitors[0].get("homeAway") == "home"
            is_regular = g.get("season", {}).get("type") == 2
            is_final = g.get("status", {}).get("type", {}).get("completed", False)
            if is_home and is_regular and is_final:
                game_date = datetime.strptime(g["date"], "%Y-%m-%dT%H:%MZ")
                days_since = (now - game_date).days
                if days_since <= 7:
                    recent_game = g
                    break
        if not recent_game:
            print("No regular season home game in last 7 days.")
            return "offseason"
        # Print details of the last game
        game_date = datetime.strptime(recent_game["date"], "%Y-%m-%dT%H:%MZ")
        print(f"Last game date: {game_date}")
        print(f"Opponent: {recent_game['competitions'][0]['competitors'][1]['team']['displayName']}")
        print(f"Final score: Rays {recent_game['competitions'][0]['competitors'][0]['score']} - {recent_game['competitions'][0]['competitors'][1]['score']} {recent_game['competitions'][0]['competitors'][1]['team']['abbreviation']}")
        # Check Rays pitcher strikeouts
        stats = None
        for c in recent_game["competitions"][0]["competitors"]:
            if c.get("team", {}).get("abbreviation") == "TB":
                stats = c.get("statistics", [])
                break
        strikeouts = 0
        if stats:
            for s in stats:
                if s.get("name") == "pitching":
                    for stat in s.get("stats", []):
                        if stat.get("name") == "strikeouts":
                            strikeouts = int(stat.get("value", 0))
        print(f"Rays pitcher strikeouts: {strikeouts}")
        days_since = (now - game_date).days
        if strikeouts >= 10 and days_since <= 5:
            return "active"
        return "not active"
    except Exception as e:
        return f"Error: {e}"

status = get_tijuana_flats_deal_status()
print("Tijuana Flats Taco & Chips deal status:", status)

Hello, cloud notebook!
No regular season home game in last 7 days.
Tijuana Flats Taco & Chips deal status: offseason
No regular season home game in last 7 days.
Tijuana Flats Taco & Chips deal status: offseason


# Save and Export Notebook

You can save your notebook using the File menu or export it in different formats (e.g., .ipynb, .pdf) using the Export option in your cloud notebook platform.

In [15]:
# Test the Papa John's Rays deal logic

def get_score(c):
    if "score" in c:
        if isinstance(c["score"], dict) and "value" in c["score"]:
            return int(c["score"]["value"])
        try:
            return int(c["score"])
        except Exception:
            return 0
    return 0

def is_completed_game(g, now):
    status_type = g.get("status", {}).get("type", {})
    state = status_type.get("state", "")
    name = status_type.get("name", "")
    game_date = datetime.strptime(g["date"], "%Y-%m-%dT%H:%MZ")
    competitors = g["competitions"][0]["competitors"]
    rays = next((c for c in competitors if c["team"]["abbreviation"] == "TB"), None)
    opponent = next((c for c in competitors if c["team"]["abbreviation"] != "TB"), None)
    rays_score = get_score(rays)
    opp_score = get_score(opponent)
    # Consider completed if:
    # - date is before now
    # - status.type.state == 'post' or status.type.name == 'STATUS_FINAL'
    # - scores are present and date is before now
    if game_date <= now and (state == "post" or name == "STATUS_FINAL" or (rays_score is not None and opp_score is not None)):
        return True
    return False

def get_papajohns_rays_deal_status():
    api_url = "https://site.api.espn.com/apis/site/v2/sports/baseball/mlb/teams/tb/schedule"
    try:
        response = requests.get(api_url)
        data = response.json()
        games = data.get("events", [])
        now = datetime.utcnow()
        print(f"Current UTC date: {now}")
        # Debug: print all games in last 14 days
        print("\nAll games in last 14 days:")
        for g in reversed(games):
            try:
                game_date = datetime.strptime(g["date"], "%Y-%m-%dT%H:%MZ")
                days_since = (now - game_date).days
                if days_since <= 14 and days_since >= 0:
                    competitors = g["competitions"][0]["competitors"]
                    rays = next((c for c in competitors if c["team"]["abbreviation"] == "TB"), None)
                    opponent = next((c for c in competitors if c["team"]["abbreviation"] != "TB"), None)
                    rays_score = get_score(rays)
                    opp_score = get_score(opponent)
                    opp_name = opponent["team"]["displayName"] if opponent else "N/A"
                    status_obj = g.get("status", {}).get("type", {})
                    completed = is_completed_game(g, now)
                    status_name = status_obj.get("name", "N/A")
                    season_type = g.get("seasonType", {}).get("type", g.get("season", {}).get("type", "N/A"))
                    print(f"Date: {game_date}, Opponent: {opp_name}, Rays: {rays_score}, Opponent: {opp_score}, Completed: {completed}, Status: {status_name}, SeasonType: {season_type}")
            except Exception as e:
                print(f"Error parsing game: {e}")
        # Find the most recent completed regular season game
        most_recent_completed = None
        for g in reversed(games):
            season_type = g.get("seasonType", {}).get("type", g.get("season", {}).get("type", None))
            is_regular = season_type == 2 or season_type == "2"
            game_date = datetime.strptime(g["date"], "%Y-%m-%dT%H:%MZ")
            if is_regular and is_completed_game(g, now):
                most_recent_completed = g
                break
        if most_recent_completed:
            competitors = most_recent_completed["competitions"][0]["competitors"]
            game_date = datetime.strptime(most_recent_completed["date"], "%Y-%m-%dT%H:%MZ")
            rays = next((c for c in competitors if c["team"]["abbreviation"] == "TB"), None)
            opponent = next((c for c in competitors if c["team"]["abbreviation"] != "TB"), None)
            rays_score = get_score(rays)
            opp_score = get_score(opponent)
            opp_name = opponent["team"]["displayName"] if opponent else "N/A"
            opp_abbr = opponent["team"]["abbreviation"] if opponent else "N/A"
            print("\nMost recent completed regular season game:")
            print(f"Date: {game_date}")
            print(f"Opponent: {opp_name}")
            print(f"Final score: Rays {rays_score} - {opp_score} {opp_abbr}")
        else:
            print("\nNo completed regular season games found.")
        # Track all qualifying games in last 7 days
        qualifying_games = []
        for g in reversed(games):
            season_type = g.get("seasonType", {}).get("type", g.get("season", {}).get("type", None))
            is_regular = season_type == 2 or season_type == "2"
            game_date = datetime.strptime(g["date"], "%Y-%m-%dT%H:%MZ")
            if is_regular and is_completed_game(g, now):
                competitors = g["competitions"][0]["competitors"]
                rays = next((c for c in competitors if c["team"]["abbreviation"] == "TB"), None)
                opponent = next((c for c in competitors if c["team"]["abbreviation"] != "TB"), None)
                rays_score = get_score(rays)
                opp_score = get_score(opponent)
                opp_name = opponent["team"]["displayName"] if opponent else "N/A"
                days_since = (now - game_date).days
                if rays_score >= 6 and days_since <= 7:
                    qualifying_games.append({"date": game_date, "score": rays_score, "opponent": opp_name, "opp_score": opp_score})
        if not qualifying_games:
            print("\nNo qualifying game in last 7 days.")
        else:
            print("\nQualifying games in last 7 days where Rays scored 6+ runs:")
            for qg in qualifying_games:
                print(f"Date: {qg['date']}, Opponent: {qg['opponent']}, Rays: {qg['score']}, Opponent: {qg['opp_score']}")
        # Now apply the deal logic as before
        qualifying_game = qualifying_games[0] if qualifying_games else None
        prev_qualifying_game = qualifying_games[1] if len(qualifying_games) > 1 else None
        if not qualifying_game:
            return "offseason"
        q_date = qualifying_game["date"]
        today = now
        yesterday = today - timedelta(days=1)
        is_yesterday = q_date.date() == yesterday.date()
        is_today = q_date.date() == today.date()
        if is_yesterday:
            if prev_qualifying_game:
                prev_date = prev_qualifying_game["date"]
                if prev_date.date() == yesterday.date():
                    return "active"
            return "active"
        if is_today:
            # Calculate hours until midnight UTC
            midnight_utc = datetime.combine(today.date() + timedelta(days=1), datetime.min.time())
            hours_until = (midnight_utc - today).total_seconds() / 3600
            print(f"\nDeal will be activated in {hours_until:.1f} hours (at midnight UTC)")
            return "not active"
        return "not active"
    except Exception as e:
        return f"Error: {e}"

status = get_papajohns_rays_deal_status()
print("Papa John's Rays deal status:", status)

Current UTC date: 2025-08-14 20:05:30.726073

All games in last 14 days:
Date: 2025-08-14 02:05:00, Opponent: Athletics, Rays: 8, Opponent: 2, Completed: True, Status: N/A, SeasonType: 2
Date: 2025-08-13 02:05:00, Opponent: Athletics, Rays: 0, Opponent: 6, Completed: True, Status: N/A, SeasonType: 2
Date: 2025-08-12 02:05:00, Opponent: Athletics, Rays: 7, Opponent: 4, Completed: True, Status: N/A, SeasonType: 2
Date: 2025-08-10 20:10:00, Opponent: Seattle Mariners, Rays: 3, Opponent: 6, Completed: True, Status: N/A, SeasonType: 2
Date: 2025-08-10 01:40:00, Opponent: Seattle Mariners, Rays: 4, Opponent: 7, Completed: True, Status: N/A, SeasonType: 2
Date: 2025-08-09 01:40:00, Opponent: Seattle Mariners, Rays: 2, Opponent: 3, Completed: True, Status: N/A, SeasonType: 2
Date: 2025-08-06 20:07:00, Opponent: Los Angeles Angels, Rays: 5, Opponent: 4, Completed: True, Status: N/A, SeasonType: 2
Date: 2025-08-06 01:38:00, Opponent: Los Angeles Angels, Rays: 7, Opponent: 3, Completed: True, Sta